<a href="https://colab.research.google.com/github/jyoon0319/2023S/blob/main/Grad_HW_Crawling_from_MultipleVideos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://velog.io/@nellholic108/%EC%9C%A0%ED%8A%9C%EB%B8%8C-API-%EB%8C%93%EA%B8%80-%EA%B0%80%EC%A0%B8%EC%98%A4%EA%B8%B0-%ED%95%A8%EC%88%98

In [ ]:
# 필요 라이브러리 import

## 데이터 프레임 다루기 위해
import pandas as pd

## 구글 api(유튜브) 연결
from googleapiclient.discovery import build

## 진행상황 파악
from tqdm import tqdm

In [ ]:
from urllib.parse import urlparse, parse_qs

def collect_comments(video_list, result_list, api_key):
    api_obj = build('youtube', 'v3', developerKey=api_key)
    shape_count = []
    
    for video_url in tqdm(video_list, desc="Collecting Comments from YouTube..."):
        video_id = parse_qs(urlparse(video_url).query)['v'][0]
        response = api_obj.commentThreads().list(
            part='snippet,replies',
            videoId=video_id,
            maxResults=100
        ).execute()
        
        comments = []
        while response:
            for item in response['items']:
                comment = item['snippet']['topLevelComment']['snippet']
                comments.append([comment['textDisplay'], comment['publishedAt'], comment['likeCount']])

                if item['snippet']['totalReplyCount'] > 0:
                    for reply_item in item['replies']['comments']:
                        reply = reply_item['snippet']
                        comments.append([reply['textDisplay'], reply['publishedAt'], reply['likeCount']])
                        
            if 'nextPageToken' in response:
                response = api_obj.commentThreads().list(
                    part='snippet,replies',
                    videoId=video_id,
                    pageToken=response['nextPageToken'],
                    maxResults=100
                ).execute()
            else:
                break
        
        df = pd.DataFrame(comments, columns=["comment", "datetime", "like_count"])
        df = df.dropna(axis=0)
        
        print(f"{video_id}: {df.shape}")
        shape_count.append(df.shape[0])
        result_list.append(df)
    
    result_list = pd.concat(result_list, ignore_index=True)
    
    if sum(shape_count) == result_list.shape[0]:
        print("The total count matches.")
        return result_list
    else:
        print(f"Sum: {sum(shape_count)} / {result_list.shape[0]}. The counts do not match.")
        return result_list

Your API: ?? (+ video urls)

In [ ]:
#collect_comments (video_list, result_list, api_key)
result_list = []
video_list = ["https://www.youtube.com/watch?v=614jiKgMhwg&t=2s", "https://www.youtube.com/watch?v=4Ze6O7CJxZY&t=46s", "https://www.youtube.com/watch?v=90rgSW3dDVM&t=62s", "https://www.youtube.com/watch?v=Do5asVYNQoQ&t=33s", "https://www.youtube.com/watch?v=g6pquvaqObo"]
api_key = ''
df = collect_comments (video_list, result_list, api_key)

614jiKgMhwg: (834, 3)


4Ze6O7CJxZY: (654, 3)
90rgSW3dDVM: (166, 3)


Do5asVYNQoQ: (498, 3)
g6pquvaqObo: (20, 3)
The total count matches.


2. 언어별 엑셀로 저장하기

In [ ]:
# Separate different language into different spread sheets
# Installation
import pandas as pd
! pip install langdetect
from langdetect import detect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=763c020da7aaa3744bc52eb521f49243592b61411b885dd663808c44ee01469b
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
df.head()

,comment,datetime,like_count
0,Could be a much devotion and respect to your m...,2023-05-29T17:51:55Z,0
1,Sadly his people distrust his relationship wit...,2023-05-21T07:44:31Z,0
2,Hello im korean!<br>He was able to be presiden...,2023-05-30T06:50:57Z,0
3,99% All south korean men go to military and we...,2023-05-21T02:49:59Z,1
4,is he very much like an ship ? !?? to usa ...,2023-05-19T19:35:40Z,0


In [ ]:
# df['comment'] 언어탐지하여 나눠서 저장











<ipython-input-22-b26d74da74ba>:34: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  excel_file.save()


3. 워드클라우드

4. 토픽모델링 (답이 안 나올지도...)

5. 감성분석